In [ ]:
# Cell 1: Imports & Paths
import json, random, os

# Paths (relative to this notebook's folder)
DATA_DIR     = os.path.join('..','book_data')
META_IN      = os.path.join(DATA_DIR, 'meta_Books.json')
REVIEWS_IN   = os.path.join(DATA_DIR, 'Books.json')
META_OUT     = 'filtered_meta_Books.json'
REVIEWS_OUT  = 'filtered_Books.json'

print("Metadata input exists:", os.path.exists(META_IN))
print("Reviews input exists: ", os.path.exists(REVIEWS_IN))

In [ ]:
# Cell 2: Peek at a metadata line to confirm schema
with open(META_IN) as f:
    sample = json.loads(f.readline())
        
print("Keys in metadata sample:", sample.keys())
print("Sample categories field:", sample.get('categories') or sample.get('category'))


In [ ]:
# Cell 3: Configuration & Collect genres → ASINS

genre_to_asins = {}
with open(META_IN,'r',encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        asin = data.get('asin')
        cats = data.get('categories', []) or data.get('category', [])
        # skip if no categories
        if not cats:
            continue
        # flatten and map
        for genre in cats:
            genre_to_asins.setdefault(genre, set()).add(asin)

print("Total distinct genres found:", len(genre_to_asins))


In [ ]:
# Cell 4: Sample genres + ASINs
GENRES_TO_SELECT = 100
ENTRIES_PER_GENRE = 2000
all_genres = list(genre_to_asins)
selected_genres = random.sample(all_genres, min(GENRES_TO_SELECT, len(all_genres)))

sampled_asins = set()
for genre in selected_genres:
    asins = list(genre_to_asins[genre])
    sampled = random.sample(asins, min(ENTRIES_PER_GENRE, len(asins)))
    sampled_asins.update(sampled)
print("Selected genres:", selected_genres)
print("Total ASINs sampled:", len(sampled_asins))


In [ ]:
# Cell 5: Write filtered metadata
with open(META_IN,'r',encoding='utf-8')  as infile, open(META_OUT, 'w') as outfile:
    count = 0
    for line in infile:
        data = json.loads(line)
        if data.get('asin') in sampled_asins:
            outfile.write(json.dumps(data) + "\n")
            count += 1
print(f"Wrote {count} metadata lines to {META_OUT}")


In [ ]:
# Cell 6: Write filtered reviews
with open(REVIEWS_IN,'r',encoding='utf-8')  as infile, open(REVIEWS_OUT, 'w') as outfile:
    count = 0
    for line in infile:
        data = json.loads(line)
        if data.get('asin') in sampled_asins:
            outfile.write(json.dumps(data) + "\n")
            count += 1
print(f"Wrote {count} review lines to {REVIEWS_OUT}")
